In [1]:
# %pip install lionagi

from lionagi import __version__
from lionfuncs import time

print("lionagi version \t", __version__)
print("last edited date \t", time(type_="iso", sep="T").split("T")[0])

lionagi version 	 0.3.5
last edited date 	 2024-10-13


## LionAGI - 103

Reflective Iterative editing of a piece of text

In [2]:
from IPython.display import display, Markdown


def printmd(string):
    display(Markdown(string))

In [3]:
from lionagi import iModel, Branch

gpt4o_config = {
    "provider": "openai",
    "model": "gpt-4o",
    "api_key_schema": "OPENAI_API_KEY",
}

gpt4o = iModel(**gpt4o_config)

Introducing custom logic to LLM flow

---

Now we can do something more complex other than having LLMs following through a pre-defined flow. We can introduce custom logic to the LLM flow. 

But first let's take a look at `structured_output`


LLM works by taking a bunch of texts and generate a bunch of texts. Natural Language Processing (NLP) isn't necessarily easy to to help us make use of the response, and one way to mitigate that is **structured output**

going back to the comedian example

In [4]:
comedian_system_message = "a respected zoo keeper, also a comedian who knows very well about indian post modern literature"
critic_system_message = "As an a young professor, you are trying to get into the prestigious circle of literature criticism."

In [5]:
branch = Branch(system=comedian_system_message, imodel=gpt4o)

In [6]:
prompt1 = "Tell a joke: a blue whale met a big white shark in a bar, then started dancing."

prompt2 = "Tell a joke: an elephant knocked off a banana tree."

response = await branch.chat(prompt1)
printmd(response)

Sure, here's a light-hearted take:

A blue whale and a big white shark walk into a bar. The bartender looks up and says, "Wow, we don't get many of your kind in here!" The blue whale replies, "Well, we heard the dance floor was big enough for us to really make a splash!"

The shark nods and adds, "Yeah, and I heard the drinks are killer!"

Then they both hit the dance floor, and the whale starts doing the wave while the shark shows off its fin-tastic moves. The crowd goes wild, and someone shouts, "Now that's what I call a whale of a time!"

And the bartender just shakes his head, thinking, "I guess this is what happens when you mix a little liquid courage with a lot of ocean motion!"

now intead of asking comedian to give a plain joke, let's instruct it to also give a few other things, by giving an example to the comedian.

In [7]:
example = {
    "category": "jokes",
    "topic": "assigning personalities to animals",
    "interpretation": "The..... therefore.....",
    "joke": "...",
    "rating": "an integer from 1 to 10",
}

response = await branch.chat(
    instruction=prompt2,
    requested_fields=example,
)

In [8]:
from lionfuncs import as_readable_json

str_ = as_readable_json(response)

print(str_)

{
    "category": "jokes",
    "topic": "assigning personalities to animals",
    "interpretation": "The joke plays on the idea of elephants being large and clumsy, therefore humorously exaggerating their interactions with their environment.",
    "joke": "Why did the elephant knock off the banana tree? Because it wanted to make a smoothie, but forgot to peel out first!",
    "rating": 7
}


In [9]:
response["rating"]

7

now we can see that the comedian can give us a joke, a joke type, and a joke rating. By making the output a `dict` we can effectively use the output of the comedian. 

We asked it to give itself a rating, here is an example to use it: 

ask comedian to repeat fixing this joke until it reaches **8/10**

In [10]:
from lionfuncs import to_num, to_dict


async def write_a_scored_joke(prompt, idx):
    response = await branch.chat(
        instruction=prompt,
        requested_fields=example,
    )
    response = to_dict(response)
    response["rating"] = to_num(response.get("rating", 0))

    str_ = f"**Joke No.{idx+1}**:  \n\n " + response["joke"]
    str_ += "\n\n **Interpretation**:  \n\n " + response["interpretation"]
    str_ += "\n\n **Rating**:  \n\n " + str(response["rating"])

    printmd(str_)

    return response


async def reflective_joke(prompt, max_attempts=3):

    response = await write_a_scored_joke(prompt, 0)
    ctr = 1

    while response["rating"] < 8 and ctr < max_attempts:
        printmd(" \n **Not a great joke, trying again** \n\n --- \n ")
        response = await write_a_scored_joke(prompt, ctr)
        ctr += 1

    if response["rating"] >= 8:
        printmd(
            "  \n **Great joke!** \n\n --- \n End of the joke session \n\n "
        )
    else:
        printmd(
            "  \n **No great jokes found** \n\n --- \n End of the joke session \n\n "
        )

In [11]:
await reflective_joke(prompt2)

**Joke No.1**:  

 Why did the elephant knock off the banana tree? It was just trying to reach for a snack, but ended up going bananas!

 **Interpretation**:  

 The joke highlights the elephant's size and strength, therefore humorously suggesting it unintentionally causes chaos.

 **Rating**:  

 6.0

 
 **Not a great joke, trying again** 

 --- 
 

**Joke No.2**:  

 Why did the elephant knock off the banana tree? It just wanted to shake things up a bit!

 **Interpretation**:  

 The joke plays on the elephant's notorious size and strength, therefore humorously suggesting that even a simple action can lead to unintended consequences.

 **Rating**:  

 7.0

 
 **Not a great joke, trying again** 

 --- 
 

**Joke No.3**:  

 Why did the elephant knock off a banana tree? It was just trying to pick a banana, but ended up with the whole bunch!

 **Interpretation**:  

 The joke uses the elephant's size and strength to create a humorous scenario, therefore suggesting that even a gentle action can have big effects.

 **Rating**:  

 8.0

  
 **Great joke!** 

 --- 
 End of the joke session 

 